# TODO
1. replicate all the other report graphs
    - graphs to create: 
        - revision.png
        - time_at_editor.png
        - main_issues.png
        - download_histograms.png
        - downloads.png
2. create a dashboard containing all the graphs

In [19]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

## Revision.png & time_at_editor.png

In [2]:
analysis_df = pd.read_stata('data/git-events.dta')

/tmp/ipykernel_11235/973408826.py:1: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  analysis_df = pd.read_stata('data/git-events.dta')


In [3]:
analysis_df = analysis_df.sort_values(by=['MS', 'numeric_date'])
analysis_df['spell'] = (analysis_df['numeric_date']-analysis_df['numeric_date'].shift(1))/3600/24
analysis_df['matching'] = (analysis_df['MS'] == analysis_df['MS'].shift(1))
make_missing = analysis_df.index[analysis_df['matching']==False]
analysis_df.loc[make_missing, 'spell'] = np.NaN
analysis_df['at_editor'] = (analysis_df['branch_imputed'].shift(1) == analysis_df['branch_imputed']) | (analysis_df['branch_imputed'].shift(1) == 'author')
analysis_df['change'] = -(analysis_df['at_editor'].shift(1)==analysis_df['at_editor'])
analysis_df['spell_id'] = analysis_df.groupby('MS')['change'].transform(pd.Series.cumsum)

In [4]:
collapsed = analysis_df.groupby(by=['MS', 'accepted_at', 'spell_id', 'at_editor']).sum().reset_index()
collapsed = collapsed[['MS', 'accepted_at', 'spell_id', 'at_editor', 'spell']]
collapsed = collapsed[collapsed['spell_id'] != 1]
collapsed['spell_id'] = np.floor((collapsed['spell_id']-1)/2)
collapsed['spell_id'] = collapsed['spell_id'].astype(int)
collapsed = collapsed.pivot(index=['MS', 'spell_id'], columns='at_editor', values=['spell', 'accepted_at']).reset_index()
collapsed.columns = ['MS', 'revision','time_at_author', 'time_at_editor', 'accepted_at0', 'accepted_at']
collapsed.drop('accepted_at0',axis=1,inplace=True)
collapsed['max_revision'] = collapsed.groupby('MS')['revision'].transform(pd.Series.max)

/tmp/ipykernel_11235/1264683990.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  collapsed = analysis_df.groupby(by=['MS', 'accepted_at', 'spell_id', 'at_editor']).sum().reset_index()


In [41]:
time_at_editor_chart = go.Figure()
time_at_editor_chart.add_trace(go.Histogram(
        x=collapsed[collapsed['revision']==0]['time_at_editor'],
        hovertemplate="%{y}",
        name='First submission',
        )
    )
time_at_editor_chart.add_trace(go.Histogram(
        x=collapsed[collapsed['revision']>0]['time_at_editor'],
        hovertemplate="%{y}",
        name='Revision',
    )
)
time_at_editor_chart.update_layout(
    title={
            "text": f"Time at editor",
            },
    font = dict(
            size = 14
            ),
    showlegend = False,
    hoverlabel = dict(
            font_size = 14,
            font_family = "Rockwell"
        ),
    barmode='overlay',
    xaxis_title_text='Days at editor',
    yaxis_title_text='Frequency',
)
time_at_editor_chart.update_traces(opacity=0.5)
time_at_editor_chart.show()

In [35]:
revisions = go.Figure(data = go.Histogram(
            x=collapsed[collapsed['revision']==0]['max_revision'],
            hovertemplate="%{y}<extra></extra>"
        )
    )
revisions_chart = go.Figure(data=revisions)
revisions_chart.update_layout(
    title={
            "text": f"Number of revision round for accepted packages",
            },
    font = dict(
            size = 14
            ),
    showlegend = False,
    hoverlabel = dict(
            font_size = 14,
            font_family = "Rockwell"
        ),
    barmode='overlay',
    xaxis_title_text='Accepted revison',
    yaxis_title_text='Frequency',
)
revisions_chart.update_traces(opacity=0.5)
revisions_chart.show()